## Stage 2: Tool Use / Function Calling

It's time to give agent the ability to call real-world functions like fetching data or executing code when needed, not just describe what it would do.

### Concept Behind Tool Use:

LLMs are good at reasoning, but they can't directly:

- access APIs

- run Python

- browse the web, unless we build that in.


In [ ]:
%pip install together python-dotenv

In [16]:
from together import Together
from dotenv import load_dotenv
import time
import json

In [17]:
from together import Together
load_dotenv()
client = Together()

# step 1 - Define a Tool
tools = {
    "add": lambda a, b: a + b
}

def run_agent(goal, max_steps=5):
    memory = ""
    for step in range(max_steps):
        # Call the LLM
        response = client.chat.completions.create(
            model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",   
            messages=[{"role": "system", "content": "You are a helpful agent."},
                      {"role": "user", "content": goal},
                      {"role": "assistant", "content": memory}],
            tools=[{
                "type": "function",
                "function": {
                    "name": "add",
                    "description": "Adds two numbers",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "a": {"type": "number"},
                            "b": {"type": "number"}
                        },
                        "required": ["a", "b"]
                    }
                }
            }]
        )

        msg = response.choices[0].message
        
        # If it's a tool call
        if msg.tool_calls:
            for tc in msg.tool_calls:
                fn = tc.function.name
                args = eval(tc.function.arguments)
                result = tools[fn](**args)

                # feed result back to LLM
                memory += f"\nTool {fn} returned: {result}"
                print(f"Tool call: {fn}({args})\nResult = {result}")
        else:
            print("Final Answer:", msg.content)
            break
            
        time.sleep(2)

# run_agent("Can you add 4 + 6 and also 7 + 8?",max_steps=1)
run_agent("What is the sum of 25 and 30? Also, can you do 2 plus 5?",max_steps=2)

Tool call: add({'a': 25, 'b': 30})
Result = 55
Tool call: add({'a': 25, 'b': 30})
Result = 55
